In [1]:
import json
from dataclasses import dataclass
from typing import List, Optional

@dataclass
class PatientCervicalData:
    # Basic demographic and screening selection:
    question1: List[str]            # Screening choices (must include "Item 2" for cervical cancer)
    question2: Optional[int] = None   # Age
    question3: Optional[str] = None   # Gender ("Item 1" means زن / female)
    
    # Additional questions for cervical screening:
    question63: Optional[str] = None  # Prior cervical screening: 
                                      #   "Item 1" = خیر (No), "Item 2" = بله (Yes), "Item 3" = نمی دانم (Don't know)
    question64: Optional[str] = None  # If question63 == "Item 2", asks if prior screening result was abnormal:
                                      #   "Item 1" = خیر, "Item 2" = بله, "Item 3" = نمی دانم
    question62: Optional[str] = None  # HPV vaccination status: 
                                      #   "Item 1" = خیر, "Item 2" = بله, "Item 3" = نمی دانم
    
    # Immunocompetence questions:
    question11: Optional[str] = None  # HIV history: "Item 1" = خیر, "Item 2" = بله, "Item 3" = نمی دانم
    question61: Optional[str] = None  # Other immune deficiency: "Item 1" = خیر, "Item 2" = بله, "Item 3" = نمی دانم

def get_cervical_cancer_screening_recommendation(patient: PatientCervicalData) -> str:
    """
    Returns a recommendation message regarding cervical cancer screening that exactly states
    whether the patient is indicated for screening and, if so, which screening strategies are acceptable.

    The logic is as follows:
    
    1. Required questions: screening selection (question1), age (question2), gender (question3),
       prior cervical screening (question63), HPV vaccination status (question62), and immunocompetence 
       (question11 and question61). Additionally, if the patient has had prior screening (question63 = "Item 2"),
       then question64 (abnormal result history) is required.
       
    2. The patient must have selected cervical cancer screening ("Item 2" in question1) and be female (question3 = "Item 1").
    
    3. If the patient is immunocompromised (question11 or question61 = "Item 2"), routine age-based screening 
       for average risk does not apply.
       
    4. If the patient has had prior cervical screening and reports an abnormal result (question63 = "Item 2" and 
       question64 = "Item 2"), she should be managed with active surveillance.
       
    5. Otherwise, age-based recommendations are:
       - Age < 21: Screening is not indicated.
       - Age 21 to 29: Pap test alone every three years.
       - Age 30 to 65: Acceptable strategies include:
             • Primary HPV testing every five years, or
             • Co-testing (Pap and HPV testing) every five years, or
             • Pap test alone every three years.
       - Age > 65: If prior screening is unknown or considered inadequate based on available responses,
         screening is indicated. One approach is co-testing annually for three years, then every five years.
         *Note:* The determination of "adequate and normal" prior screening requires gynecologic expertise.
         Please refer to a gynecologist for further evaluation.
    """
    
    # Check that all required fields are provided:
    missing_fields = []
    if not patient.question1:
        missing_fields.append("question1 (screening choices)")
    if patient.question2 is None:
        missing_fields.append("question2 (age)")
    if not patient.question3:
        missing_fields.append("question3 (gender)")
    if not patient.question63:
        missing_fields.append("question63 (prior cervical screening)")
    if not patient.question62:
        missing_fields.append("question62 (HPV vaccination status)")
    if not patient.question11:
        missing_fields.append("question11 (HIV status)")
    if not patient.question61:
        missing_fields.append("question61 (immune deficiency status)")
    # When the patient indicates prior screening, abnormal result history is required.
    if patient.question63 == "Item 2" and not patient.question64:
        missing_fields.append("question64 (abnormal screening result history)")
    
    if missing_fields:
        return "Missing required data: " + ", ".join(missing_fields)
    
    # Confirm the patient selected cervical cancer screening.
    if "Item 2" not in patient.question1:
        return "Patient is not indicated for cervical cancer screening."
    
    # Confirm the patient is female.
    if patient.question3 != "Item 1":
        return "Patient is not indicated for cervical cancer screening."
    
    # Check immunocompetence:
    if patient.question11 == "Item 2" or patient.question61 == "Item 2":
        return ("Patient is not indicated for routine cervical cancer screening due to immunocompromised status. "
                "Alternative management or active surveillance is recommended.")
    
    # Check for abnormal prior screening:
    if patient.question63 == "Item 2":
        if patient.question64 == "Item 2":
            return ("Patient is not indicated for routine cervical cancer screening as she has a history of abnormal screening results. "
                    "Active surveillance is recommended.")
    
    # Age-based screening recommendations:
    age = patient.question2
    if age < 21:
        return "Patient is not indicated for cervical cancer screening."
    elif 21 <= age < 30:
        return "Patient is indicated for cervical cancer screening: Pap test alone every three years."
    elif 30 <= age <= 65:
        return ("Patient is indicated for cervical cancer screening. Acceptable strategies include: "
                "Primary HPV testing every five years, or co-testing (Pap and HPV testing) every five years, "
                "or Pap test alone every three years.")
    else:  # age > 65
        # For patients over 65, check the prior screening response.
        # "Item 1" (No) or "Item 3" (Don't know) indicates that prior screening is unknown/inadequate.
        if patient.question63 in ["Item 1", "Item 3"]:
            return ("For patients over 65 with unknown or inadequate prior screening (based on available responses), "
                    "screening is indicated. One approach is co-testing annually for three years, then every five years. "
                    "Note: The definition of 'adequate and normal' prior screening requires gynecologic expertise. "
                    "Please refer to a gynecologist for further evaluation.")
        else:  # patient.question63 == "Item 2"
            return ("For patients over 65 with a history of prior screening, if prior screening is known to be adequate and normal, "
                    "screening may be discontinued. However Determining whether prior screening is adequate and normal requires gynecologic expertise. "
                    "Please refer to a gynecologist for further evaluation.")

# Example usage:
if __name__ == "__main__":
    # Example patient data:
    sample_response = {
        "question1": ["Item 2", "Item 5"],  # Patient selected cervical cancer screening ("Item 2")
        "question2": 70,                    # Age 70
        "question3": "Item 1",              # Female
        "question63": "Item 3",             # Unknown prior cervical screening ("نمی دانم")
        "question62": "Item 2",             # Has had HPV vaccination (بله)
        "question11": "Item 1",             # No history of HIV (خیر)
        "question61": "Item 1"              # No history of immune deficiency (خیر)
        # question64 is not needed since question63 is not "Item 2"
    }
    
    patient_data = PatientCervicalData(
        question1=sample_response.get("question1", []),
        question2=sample_response.get("question2"),
        question3=sample_response.get("question3"),
        question63=sample_response.get("question63"),
        question64=sample_response.get("question64"),
        question62=sample_response.get("question62"),
        question11=sample_response.get("question11"),
        question61=sample_response.get("question61")
    )
    
    recommendation = get_cervical_cancer_screening_recommendation(patient_data)
    print(recommendation)


For patients over 65 with unknown or inadequate prior screening (based on available responses), screening is indicated. One approach is co-testing annually for three years, then every five years. Note: The definition of 'adequate and normal' prior screening requires gynecologic expertise. Please refer to a gynecologist for further evaluation.
